# Pokemon Analysis - Some Questions and Answers

In [2]:
import pandas as pd
import numpy as np

In [3]:
# These questions can be answered with two datasets:
pokemon = pd.read_csv('Pokemon.csv')
types = pd.read_csv('Types.csv')

In [4]:
# The pokemon dataframe contains all pokemons (first 7 generations) and their stats
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
# The types dataframe contains type effectiveness information
types

,Attacking,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
0,Normal,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5,1.0
1,Fire,1,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0,1.0
2,Water,1,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0,1.0
3,Electric,1,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0
4,Grass,1,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5,1.0
5,Ice,1,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0
6,Fighting,2,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0,0.5
7,Poison,1,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0,2.0
8,Ground,1,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0,1.0
9,Flying,1,1.0,1.0,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,0.5,1.0


# Question 1: How many unique type combinations are there?


Let $V$ be a one-dimensional vector containing all pokemon types. Then, all type combinations are uniquely defined by the upper (or lower) triangle of the outer product of this vector. 

$\text{all_combos} = V.V^T$

Note1: the diagonal of this matrix is all monotypes. there are 18 
<br>
Note2: the upper and lower triangle (discluding the diagonal) are symmetric with respect to combinations, but non-unique with respect to permutations. The permutations can represent primary and secondary typings of dual type pokemons. For example, all_combos[0,1] = NormalFire and all_combos[1,0] = FireNormal. Since I am not aware of primary/secondary typings having much impact, let us only care about unique combinations.


In [6]:
types_vector = np.asarray(types['Attacking'])
all_permutations = types_vector[:,None]+types_vector
all_combos = set(np.triu(all_permutations).flatten())
all_combos.remove(0) # some extra fluff from np.triu


In [7]:
print("Answer: There are exactly %d type combinations in Pokemon" % (len(all_combos)))

Answer: There are exactly 171 type combinations in Pokemon


Obviously, it was not necessary to compute all unique combinations explicitly, since the answer is a triangular number: 
<br>
$N * (N+1) / 2 = 18 * 19 / 2 = 171$

# Question 4: What is the minimum number of unique typed moves to be super effective against all theoretical pokemon typings?

This question is hugely interesting, since it could lead to a moveset that has 100% super effective coverage

In [8]:
# we are going to need a helper function to solve this problem
def typeCombinationSplitter(combo: str):
    # splits the combination typing into component types
    t = set([])
    for pokemon_type in types_vector:
        if pokemon_type in combo:
            t.add(pokemon_type)
    # note: we lexographically sort here, to aid in 
    # checking for uniqueness later on.
    return tuple(sorted(list(t)))


In [9]:
# to confirm out helper function is working appropriately
for combo in all_combos:
    print(combo, typeCombinationSplitter(combo))

GroundRock ('Ground', 'Rock')
NormalSteel ('Normal', 'Steel')
NormalPoison ('Normal', 'Poison')
NormalRock ('Normal', 'Rock')
FightingPsychic ('Fighting', 'Psychic')
PoisonGround ('Ground', 'Poison')
NormalElectric ('Electric', 'Normal')
DarkFairy ('Dark', 'Fairy')
WaterPoison ('Poison', 'Water')
NormalPsychic ('Normal', 'Psychic')
FireGround ('Fire', 'Ground')
FlyingSteel ('Flying', 'Steel')
WaterDark ('Dark', 'Water')
GrassGround ('Grass', 'Ground')
GrassDark ('Dark', 'Grass')
GrassSteel ('Grass', 'Steel')
FightingFlying ('Fighting', 'Flying')
GhostDragon ('Dragon', 'Ghost')
NormalFighting ('Fighting', 'Normal')
WaterPsychic ('Psychic', 'Water')
IceGhost ('Ghost', 'Ice')
GrassFighting ('Fighting', 'Grass')
IceFairy ('Fairy', 'Ice')
GhostFairy ('Fairy', 'Ghost')
DragonSteel ('Dragon', 'Steel')
GrassFlying ('Flying', 'Grass')
SteelFairy ('Fairy', 'Steel')
BugBug ('Bug',)
ElectricGrass ('Electric', 'Grass')
GrassPoison ('Grass', 'Poison')
PsychicFairy ('Fairy', 'Psychic')
GrassDragon ('

In [10]:
def typeComboVulnerabilities(combo):
    # computes the vector of type vulnerabilities for this 
    # type combinations
    # TODO: remove copy.. it messed up things otherwise..
    
    if(len(combo) == 1):
        vals = types[combo[0]].copy()
    
    else:
        # it's just broadcasted product
        vals = types[combo[0]] * types[combo[1]]
    vals.index = types['Attacking']
    return vals

In [11]:
# for example
type1 = typeCombinationSplitter("BugFire")
type2 = typeCombinationSplitter("BugBug")

print(typeComboVulnerabilities(type1))
#print(typeComboVulnerabilities(type2))

Attacking
Normal      1.00
Fire        1.00
Water       2.00
Electric    1.00
Grass       0.25
Ice         0.50
Fighting    0.50
Poison      1.00
Ground      1.00
Flying      2.00
Psychic     1.00
Bug         0.50
Rock        4.00
Ghost       1.00
Dragon      1.00
Dark        1.00
Steel       0.50
Fairy       0.50
dtype: float64


In [12]:
def typePoolEffectiveness(type_pool: list):
    # returns how many pokemons this type pool 
    # would be super effective against
    count = 0
    for pokemon_typing in all_combos:
        combo = typeCombinationSplitter(pokemon_typing)
        vuln = typeComboVulnerabilities(combo)
        for movetype in type_pool:
            if vuln[movetype] >= 2:
                count += 1
                break
    return count

In [13]:
# typePoolEffectiveness will give us the number of theoretical types that 
# a given movepool will at least have one super effective move against.

# Here is a test of one of the well-known high coverage movepoools for
# Electivire: 
count = typePoolEffectiveness(['Electric', 'Ice', 'Fighting', 'Ground'])
print("Effective against %s out of 171 theoretical typings" % (count))

Effective against 135 out of 171 theoretical typings


In [14]:
from itertools import combinations
def getBestMovePoolsUsingNMoves(N):

    ans = None
    best = 0
    for type_pool_combo in combinations(types['Attacking'], N):
        val = typePoolEffectiveness(type_pool_combo)
        if val == best:
            ans.append(type_pool_combo)
        elif(val > best):
            best = val
            ans = [type_pool_combo]
    
    return best, ans
        

In [16]:
count, ans = getBestMovePoolsUsingNMoves(7)
print(count, ans)

168 [('Fire', 'Grass', 'Ground', 'Rock', 'Dark', 'Steel', 'Fairy')]


In [29]:
count, ans = getBestMovePoolsUsingNMoves(2)
print(count, ans)

109 [('Ice', 'Ground')]


In [26]:
# based on what is super effective against it
best  = [18, None]
worst = [0, None]
for pokemon_typing in all_combos:
    combo = typeCombinationSplitter(pokemon_typing)
    vuln = typeComboVulnerabilities(combo)
    supp = [val for val in vuln if val >= 2]
    n = len(supp)
    if(n < best[0]):
        best = [n, [combo]]
    elif(n == best[0]):
        best = [n, best[1] + [combo]]
    if(n > worst[0]):
        worst = [n, [combo]]
    elif(n == worst[0]):
        worst = [n, worst[1] + [combo]]
print("best: ", best)
print("worst: ", worst)

best:  [1, [('Bug', 'Steel'), ('Ghost', 'Normal'), ('Dark', 'Ghost'), ('Electric',), ('Normal',), ('Dark', 'Poison'), ('Ground', 'Water')]]
worst:  [7, [('Dark', 'Grass'), ('Psychic', 'Rock'), ('Grass', 'Ice'), ('Dark', 'Rock'), ('Grass', 'Psychic'), ('Fighting', 'Rock')]]


In [28]:
# Based on invulnerabilities
best  = [18, None]
worst = [0, None]
for pokemon_typing in all_combos:
    combo = typeCombinationSplitter(pokemon_typing)
    vuln = typeComboVulnerabilities(combo)
    supp = [val for val in vuln if val == 0.]
    n = len(supp)
    if(n < best[0]):
        best = [n, [combo]]
    elif(n == best[0]):
        best = [n, best[1] + [combo]]
    if(n > worst[0]):
        worst = [n, [combo]]
    elif(n == worst[0]):
        worst = [n, worst[1] + [combo]]
print("best: ", worst)

best:  [3, [('Fairy', 'Ghost'), ('Ghost', 'Normal'), ('Dark', 'Ghost'), ('Flying', 'Ghost'), ('Ghost', 'Steel'), ('Ghost', 'Ground')]]
